In [49]:
# groupby operation rely on a concept called "split-apply-combine"
# the "split-apply-combine" concept is also used heavily in "big data" systems that uses distributed computing 

# Basic one-variable grouped aggregation

In [1]:
import pandas as pd
import numpy as np
data = pd.read_excel(r"C:\Users\tanzh\Documents\Python\Pandas for Everyone\datasets\gapminder.xlsx")

In [51]:
avg_life_exp_by_year = data.groupby("year").lifeExp.mean()

# this group the data by year and display the average life expectancy by year
#1 the groupby statement can be though of creating a subset of each unique value of a column / unique pairs from columns

In [52]:
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [53]:
# ths below allow us to extract the unqiue values in the defined columns 
data.year.unique()

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

In [54]:
# we can create a subset of data as follows : 
y1952 = data.loc[data.year == 1952, :]

 # this create a dataframe where the entries only contain rows from where year == 1952
print(y1952)

country continent  year  lifeExp       pop    gdpPercap
0            Afghanistan      Asia  1952   28.801   8425333   779.445314
12               Albania    Europe  1952   55.230   1282697  1601.056136
24               Algeria    Africa  1952   43.077   9279525  2449.008185
36                Angola    Africa  1952   30.015   4232095  3520.610273
48             Argentina  Americas  1952   62.485  17876956  5911.315053
...                  ...       ...   ...      ...       ...          ...
1644             Vietnam      Asia  1952   40.412  26246839   605.066492
1656  West Bank and Gaza      Asia  1952   43.160   1030585  1515.592329
1668         Yemen, Rep.      Asia  1952   32.548   4963829   781.717576
1680              Zambia    Africa  1952   42.038   2672000  1147.388831
1692            Zimbabwe    Africa  1952   48.451   3080907   406.884115

[142 rows x 6 columns]


In [55]:
# the average life expectancy can be calculated as follows : 
print(y1952.lifeExp.mean())

# the groupby statement esstentially repeat the above process for every year column and return all the results in a single dataframe

49.05761971830987


# Built-in Aggregation Method

In [2]:
# describe is a built-in aggregation method

continent_lifeexp_describe = data.groupby("continent").lifeExp.describe().reset_index()
print(continent_lifeexp_describe)

continent  count       mean        std     min       25%      50%       75%  \
0    Africa  624.0  48.865330   9.150210  23.599  42.37250  47.7920  54.41150   
1  Americas  300.0  64.658737   9.345088  37.579  58.41000  67.0480  71.69950   
2      Asia  396.0  60.064903  11.864532  28.801  51.42625  61.7915  69.50525   
3    Europe  360.0  71.903686   5.433178  43.585  69.57000  72.2410  75.45050   
4   Oceania   24.0  74.326208   3.795611  69.120  71.20500  73.6650  77.55250   

      max  
0  76.442  
1  80.653  
2  82.603  
3  81.757  
4  81.235  


# Aggregation Functions

In [57]:
# For aggregation method that are not listed in the Pandas method, you can call the agg/aggregate method and pass the aggregation function you want in there
# for example : 
contin_agg = data.groupby("continent").lifeExp.aggregate(np.mean)
print(contin_agg)

# note that agg and aggregate do the same thing

continent
Africa      48.865330
Americas    64.658737
Asia        60.064903
Europe      71.903686
Oceania     74.326208
Name: lifeExp, dtype: float64


# Custom User Functions 

In [58]:
# To perform calculation that is not provided by Pandas or another library, we can write our own function
# the example below show a custom defined function for calcuating the mean

In [59]:
def mean(values) : 
    return sum(values) / len(values)

In [60]:
agg_my_mean = data.groupby("year").lifeExp.agg(mean)

In [61]:
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [62]:
# we can write functions that take multiple parameters so long as the first parameters take the series of the values from the dataframe, you can pass the other agruements as keywords into agg
# the below , we calculate the global average life expectancy and subtract it from each grouped values 

In [63]:
def mean_diff(values, diff_values): 
    mean = sum(values) / len(values)
    return (mean - diff_values)

In [64]:
global_mean = data.lifeExp.mean()

In [65]:
agg_mean_diff = data.groupby("year").lifeExp.agg(mean_diff, diff_values = global_mean)
print(agg_mean_diff)

# diff_values is a parameter in the function < mean_diff > 

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


# Multiple Functions Simultaneously 

In [66]:
# we can pass the individual functions into agg as a Python list 

In [67]:
report = data.groupby("year").lifeExp.agg([np.count_nonzero, np.mean, np.std]) # notice that the different count_nonzero, np.mean and np.std are in a list
print(report)

count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


# Using a dict in Agg -- on a dataframe

In [68]:
# there are some other ways you can apply functions in the agg method. For example you can pass a agg a Python dictionary.However the results may differ depending on whether you are aggregating directly on a DataFrame or on a Series object

# when specifying dict on a grouped DataFrame, THE KEYS ARE THE COLUMNS OF THE DATAFRAME, and the values are the function used in the aggregated calculation. 
# this approach allows you to group on one or more variables and use a different aggregation function on different columns simultaneously

gdf_dict = data.groupby("year").agg({ "lifeExp" : "mean" , "pop" : "median" , "gdpPercap" : "median"
})

print(gdf_dict)

lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


# Transform

In [69]:
# when we perform agg, we take multiple values and return a single aggregated value 
# in transformation. we take multiple values and return a one-to-one transformation of the values
# in transformation, there is no reduction of the amount of data. instead, there is an increase in the data size 

In [70]:
# z-score 
## z-score identifies the number of standard deviation from the mean of our data
## it centers our data to mean 0 with a standard deviation of 1
## this technique standardize our data and make it easier to compare different variables to each other

In [71]:
def z_score(x) : 
    return ((x - x.mean()) / x.std())

In [72]:
transform_z = data.groupby("year").lifeExp.transform(z_score)

In [73]:
data.shape # this is the number of rows and column items in our original dataset

(1704, 6)

In [74]:
transform_z.shape

(1704,)

In [75]:
# note that scipy library has its own z-score function 
from scipy.stats import zscore

sp_z_group = data.groupby("year").lifeExp.transform(zscore) # this calculates zscore on a grouped level

sp_z_nogroup = zscore(data.lifeExp) # this calculate zscore on the individual row data level (non-grouped level)

In [76]:
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [77]:
print(sp_z_group.head())

0   -1.662719
1   -1.737377
2   -1.792867
3   -1.854699
4   -1.900878
Name: lifeExp, dtype: float64


In [78]:
print(sp_z_nogroup[:5])

[-2.37533395 -2.25677417 -2.1278375  -1.97117751 -1.81103275]


In [79]:
# the grouped results are similar
# however, when we calculate the zscore outside the groupby (i.e. not at group level), we get the zscore calculated on the entire data set, not broken out by group 

# Missing Value Analysis

In [2]:
# there are various method to fill in missing values. We can replace the missing values with : 
## mean, interpolate values
## in this example, we are exploring filling the missing values based on a particular group
### we will be using the "tips" dataset froom the seaborn library

import seaborn as sns
import numpy as np

# we need to set the seed so that the results are deterministic 
np.random.seed(42)

In [3]:
# "tips"dataset show the spending amount of diners 

tips_data = sns.load_dataset("tips") # this fetch the whole dataset
tips_10 = tips_data.sample(10) # this create a subset of 10 random sample

In [82]:
tips_10.loc[np.random.permutation(tips_10.index)[:4],"total_bill"] = np.NaN
# this randomly pick 4 total_bill values and turn them into blank values

In [83]:
print(tips_10)

total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [84]:
# we may not want to use the mean value at the dataset level to compute the mean for the individual row items 
# the spending habits between male and female may and can differ significantly

# we can the use groupby statement to fill in the missing values and instead of using aggregate method, we use the transform method 

In [85]:
# first, we count the number of non-missing values by sex

count_sex = tips_10.groupby("sex").count()
print(count_sex)
# 3 of the missing values come from Male and 1 come from Female 

total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [86]:
# we then create a function to fill in the missing values based on the grouped average 
# the grouped is based on sex

def fill_na_mean(x) : # x refer to a given vector
    avg = x.mean()
    return (x.fillna(avg))

In [87]:
report_total_bill = tips_10.groupby("sex").total_bill.transform(fill_na_mean)
# this computes the average bill based on the sex
print(report_total_bill)

24     19.8200
6       8.7700
153    17.9525
211    17.9525
198    13.9300
176    17.9525
192    28.4400
124    12.4800
9      14.7800
101    15.3800
Name: total_bill, dtype: float64


In [88]:
# note that the original dataset (tips_10) still has the missing values 

tips_10["fill_total_bill"] = report_total_bill # this create a new column from the transformation process 
# you will noticed that the missing values column are now populated with values from the transformation process 

# Filter 

In [89]:
tips_data.shape

(244, 7)

In [90]:
print(tips_data["size"].value_counts())
# you might noticed that the table size of (5,6,1) are infrequent (5,4,4)
# depending on your needs, you might want to filter out the data points

# there is a total of 13 row items that you may want to filter our

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [101]:
tips_data.shape

(244, 7)

In [97]:
tips_filtered = tips_data.groupby("size").filter(lambda x: x["size"].count() > 30)

In [100]:
tips_filtered.shape

(231, 7)

In [102]:
tips_filtered["size"].value_counts()

2    156
3     38
4     37
Name: size, dtype: int64

# pandas.core.groupby.dataframegroupby object ---- GROUPS

In [92]:
# throughout this chapter, we directly chained the aggregate, transform or fliter after the groupby statement 
# however, we can save the results of the groupby before we perform these other methods
# for example, see the following codes :
grouped_10 = tips_10.groupby("sex")

In [93]:
print(grouped_10)

# this return the object and its memory location and the data type
# under the hood, nothing has been calculated because we never performed an action that require a calculation

In [94]:
print(grouped_10.groups)
# when we called for the groups, this will return the index of the dataframe back
# this is intended mainly to optimize performance 
# again, we have not calculated anything yet
# this approach allows us to save just the grouped result
# we then perform mutiple operations (aggregate/transform/filter) without having to process the groupby statement again

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'), 'Female': Int64Index([198, 124, 101], dtype='int64')}


In [95]:
tips_10.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size',
       'fill_total_bill'],
      dtype='object')

# Group Calculation involving multiple variables

In [ ]:
# If we specify the calculation method after the groupby statement, Python will perform calculation on all applicable columns

In [103]:
data.groupby("year").mean() 
# instead of data.groupby("year").lifeExp.mean()

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,1.695040e+07,3725.276046
1957,51.507401,1.876341e+07,4299.408345
1962,53.609249,2.042101e+07,4725.812342
1967,55.678290,2.265830e+07,5483.653047
1972,57.647386,2.518998e+07,6770.082815
1977,59.570157,2.767638e+07,7313.166421
1982,61.533197,3.020730e+07,7518.901673
1987,63.212613,3.303857e+07,7900.920218
1992,64.160338,3.599092e+07,8158.608521


In [104]:
tips_data.groupby("sex").mean()

,total_bill,tip,size
sex,,,
Male,20.744076,3.089618,2.630573
Female,18.056897,2.833448,2.459770


In [96]:
average_group_mean = grouped_10.mean() # this is similar to tips_10.groupby("sex").mean()
print(average_group_mean)

total_bill       tip      size  fill_total_bill
sex                                                    
Male       17.9525  2.875714  2.571429          17.9525
Female     13.9300  2.506667  2.000000          13.9300


# Selecting a sub group 

In [107]:
# we can use the get_group function to select a group. as follows : 

female = grouped_10.get_group("Female")
print(female)

total_bill   tip smoker   day    time  size  fill_total_bill
198         NaN  2.00    Yes  Thur   Lunch     2            13.93
124       12.48  2.52     No  Thur   Lunch     2            12.48
101       15.38  3.00    Yes   Fri  Dinner     2            15.38


In [110]:
# alternatively 
tips_10.loc[tips_10.sex == "Female"] # tips_10 is the original subset of the data

# note that you cannot use the above method with the a DataFrameGroupBy object such as: 
# group_10.loc[group_10.sex == "Female"] --> this will generate an error

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.93
124,12.48,2.52,Female,No,Thur,Lunch,2,12.48
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.38


# Iterating through Groups (refer to page 204/205)


In [111]:
# another benefit of saving just the groupby object is that you are able to iterate through the groups individually
for x in grouped_10 : 
    print(x)

('Male',      total_bill   tip   sex smoker   day    time  size  fill_total_bill
24        19.82  3.18  Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00  Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00  Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16  Male    Yes   Sat  Dinner     4          17.9525
176         NaN  2.00  Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56  Male    Yes  Thur   Lunch     2          28.4400
9         14.78  3.23  Male     No   Sun  Dinner     2          14.7800)
('Female',      total_bill   tip     sex smoker   day    time  size  fill_total_bill
198         NaN  2.00  Female    Yes  Thur   Lunch     2            13.93
124       12.48  2.52  Female     No  Thur   Lunch     2            12.48
101       15.38  3.00  Female    Yes   Fri  Dinner     2            15.38)


# Multiple Groups

In [4]:
# to include mutiple group in a groupby statement, place the variables in a list as follows 

x = data.groupby(["continent","country"])
x.agg({"pop" : "mean", "gdpPercap" : "std"})

pop    gdpPercap
continent country                                  
Africa    Algeria         1.987541e+07  1310.337656
          Angola          7.309390e+06  1165.900251
          Benin           4.017497e+06   159.741306
          Botswana        9.711862e+05  4178.136987
          Burkina Faso    7.548677e+06   183.430087
...                                ...          ...
Europe    Switzerland     6.384293e+06  6886.463308
          Turkey          4.590901e+07  2049.665102
          United Kingdom  5.608780e+07  7388.189399
Oceania   Australia       1.464931e+07  7815.405220
          New Zealand     3.100032e+06  4409.009167

[142 rows x 2 columns]

In [6]:
sex_time_group = tips_data.groupby(["sex","time"])
gender_spending = sex_time_group.mean()

In [7]:
type(gender_spending)

pandas.core.frame.DataFrame

In [8]:
gender_spending.columns

Index(['total_bill', 'tip', 'size'], dtype='object')

In [10]:
gender_spending.index

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])

In [132]:
# if we want to get a regular dataframe, we can call on reset_index on the results to flatten the results
report = sex_time_group.mean().reset_index()
print(report)

sex    time  total_bill       tip      size
0    Male   Lunch   18.048485  2.882121  2.363636
1    Male  Dinner   21.461452  3.144839  2.701613
2  Female   Lunch   16.339143  2.582857  2.457143
3  Female  Dinner   19.213077  3.002115  2.461538


In [ ]:
# alternatively, we can use the as_index = false parameter in the groupby method (it is TRUE by default)

In [11]:
group_pram_1 = tips_10.groupby(["sex","time"], as_index = True).mean()
print(group_pram_1)

total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [12]:
group_pram_2 = tips_10.groupby(["sex","time"], as_index = False).mean() # this will flatten the results
print(group_pram_2)

sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
